# Projet Traitement et Données Large Échelle

Zoé MARQUIS & Charlotte KRUZIC

## Comparaison de Performances entre Systèmes Relationnels et NoSQL : Étude de Cas avec le Catalogue Netflix

Dans ce projet, nous comparons les performances entre un système de base de données relationnel (SQLite) et un système NoSQL (Cassandra), en utilisant le **catalogue Netflix** comme jeu de données. Ce dataset, disponible en Open Data, offre une structure réaliste et adaptée pour explorer les différences entre ces deux approches de gestion de données. L'objectif principal est d'évaluer les performances pour des opérations d'insertion, mise à jour et requêtes complexes, tout en analysant les spécificités du système NoSQL choisi.

### Jeu de données

Le jeu de données utilisé contient des informations sur les films et séries disponibles sur Netflix, avec les attributs suivants :
    - `show_id` : Id unique pour chaque enregistrement
    - `title` : Le titre
    - `release_year` : L'année de sortie
    - `rating` : La classification
    - `duration` : La durée ou nombre de saisons (pour les séries)
    - `listed_in` : Les catégories

TODO : pourquoi ce choix

### Schéma de données

#### Structure relationnelle (SQLite)

Dans SQLite, nous utilisons une structure relationnelle classique avec les attributs définis dans une table principale, où chaque colonne correspond à un type de donnée bien spécifié. Des index sont également créés sur des colonnes clés pour optimiser les performances des requêtes fréquentes.

TODO : remplir correctement les tables par rappotr à la suite
```sql
CREATE TABLE netflix (
    show_id VARCHAR(255) PRIMARY KEY,
    title VARCHAR(255),
    release_year INT,
    rating VARCHAR(50),
    duration VARCHAR(50),
    listed_in VARCHAR(255)
);
```


#### Structure NoSQL (Cassandra)

Pour Cassandra, nous adoptons une approche orientée colonnes, en structurant les données pour des accès rapides et efficaces selon les cas d'utilisation spécifiques.
La structure NoSQL sera définie comme suit :

```sql
CREATE TABLE netflix (
    show_id TEXT PRIMARY KEY,
    title TEXT,
    release_year INT,
    rating TEXT,
    duration TEXT,
    listed_in TEXT
);
```


TODO ; checker qu'on fait bien ça par la suite



### Comparaison des schémas
Le schéma relationnel est rigide et suit une normalisation stricte pour assurer la cohérence des données, tandis que Cassandra offre plus de flexibilité dans la structure des colonnes, permettant une meilleure scalabilité horizontale.

TODO : expliciter ça

TODO : véifier

---

### Modèle de données de Cassandra

Cassandra repose sur un modèle basé sur des colonnes plutôt que sur des documents (comme MongoDB) ou des paires clé-valeur (comme Redis).

#### Keyspace
Un Keyspace dans Cassandra peut être vu comme un espace de noms, similaire à une base de données dans les systèmes relationnels. Il permet de regrouper plusieurs tables (ou Column Families). Chaque Keyspace définit des propriétés de réplication, comme le nombre de répliques des données.

#### Column Family
Les Column Families dans Cassandra ressemblent aux tables en SQL, mais avec une flexibilité accrue dans la structure des données. Une Column Family contient plusieurs colonnes, mais contrairement aux tables relationnelles, les colonnes ne doivent pas nécessairement être définies à l'avance. Elles peuvent être ajoutées dynamiquement avec des noms et des valeurs.

#### Key
Chaque ligne dans une Column Family est identifiée par une clé unique (par exemple, show_id dans votre jeu de données Netflix). Cela permet d'identifier rapidement les enregistrements sans avoir à faire de jointures complexes comme dans une base de données relationnelle.

#### Colonnes
Dans Cassandra, les données sont stockées sous forme de colonnes avec 3 composants essentiels :

- Nom : le nom de la colonne  
- Valeur : la donnée elle-même  
- Timestamp : un marqueur temporel qui permet de gérer les versions des données (utile pour les mises à jour et les conflits de réplication).  
Chaque colonne dans Cassandra est un enregistrement indépendant et peut être insérée, mise à jour ou supprimée sans affecter les autres colonnes dans la même ligne.

#### Super Colonnes
Les Super Colonnes permettent une structure encore plus flexible et imbriquée, un peu comme les données hiérarchiques. Une Super Colonne est une collection de colonnes et peut être vue comme une ligne composée de plusieurs colonnes nommées. Ce mécanisme est particulièrement utile pour des cas d'utilisation complexes où plusieurs attributs doivent être regroupés sous une même clé.

TODO : à vérifier

### Choix de Cassandra pour ce projet

Cassandra est un système de gestion de base de données NoSQL particulièrement adapté aux applications nécessitant une haute disponibilité et une scalabilité horizontale. Voici les principaux points qui justifient son choix pour ce projet, en comparaison avec d'autres systèmes NoSQL comme MongoDB ou Redis :

- **Scalabilité horizontale**  
Cassandra est conçu pour être hautement scalable, ce qui signifie qu'il peut facilement gérer de grandes quantités de données en ajoutant simplement plus de nœuds à son cluster, sans interruption de service. Contrairement à des bases comme MongoDB, qui peut rencontrer des limitations de scalabilité avec des configurations complexes de réplication et de sharding, Cassandra simplifie cette gestion.

- **Haute disponibilité et tolérance aux pannes**  
Cassandra offre une réplication native des données sur plusieurs nœuds, ce qui assure une disponibilité continue même en cas de panne de serveur ou de nœud. Ce mécanisme de réplication est essentiel dans les applications à grande échelle où la perte de données ou l'indisponibilité du service n'est pas acceptable. Ce niveau de résilience n'est pas aussi robuste dans des systèmes comme Redis, qui est plus adapté pour les caches en mémoire.

- **Modèle de données flexible**  
Cassandra adopte un modèle de données basé sur des colonnes plutôt que sur des documents (MongoDB) ou des paires clé-valeur (Redis), ce qui le rend particulièrement adapté pour les cas où la structure des données peut évoluer au fil du temps. De plus, bien que non relationnel, Cassandra permet une organisation des données en tables et index, offrant plus de structure que des systèmes comme CouchDB.

TODO : explication plus détaillée dans la cellule précédente

- **Performances optimisées pour les écritures massives**  
Cassandra est optimisé pour des opérations d'écriture à grande échelle, ce qui en fait un excellent choix pour des applications générant de grandes quantités de données en continu (comme les logs, les événements ou les analyses de séries temporelles). Contrairement à MongoDB, qui peut être plus performant pour les lectures complexes, Cassandra excelle dans les insertions et les mises à jour rapides.

- **Adapté aux cas d'utilisation distribués**
Étant un système distribué par nature, Cassandra est idéal pour des architectures distribuées et géographiquement décentralisées. Son approche de la gestion des données et de la réplication est particulièrement utile lorsque les données doivent être distribuées sur des data centers multiples, un point que d'autres bases NoSQL, comme Couchbase, ne gèrent pas aussi efficacement.

**En résumé :**
Cassandra est un excellent choix pour ce projet en raison de sa capacité à gérer des volumes de données massifs, sa scalabilité horizontale, sa tolérance aux pannes, et ses performances exceptionnelles pour les écritures. Ces caractéristiques en font un candidat idéal pour des applications nécessitant des performances de lecture/écriture rapides et une disponibilité continue, contrairement à d'autres systèmes NoSQL comme MongoDB ou Redis qui peuvent avoir des avantages dans d'autres cas d'usage.



**optimisé pour les opérations d'insertion massives.**

---
## Installer les outils

In [1]:
!pip install plotly

In [2]:
!pip install pandas
# pour manipuler des dataframes et charger les données de kaggle

In [3]:
!pip install cassandra-driver

In [4]:
# TODO à vérifier des 2 cotés
# # 🚨 sur linux 
# !apt-get update
# !apt-get install -y openjdk-11-jdk
# !apt-get install libev-dev
# !apt-get install cassandra

# # 🚨 sur mac
# !brew install openjdk@17
# !brew install libev
# !brew install cassandra


In [5]:
!pip install kagglehub
# installer kagglehub pour récupérer le dataset

In [6]:
!pip install tabulate

## Importation des données

In [7]:
import kagglehub

path = kagglehub.dataset_download("shivamb/netflix-shows")
print("Chemin vers le fichier du dataset : ", path)

Chemin vers le fichier du dataset :  /Users/zoemarquis/.cache/kagglehub/datasets/shivamb/netflix-shows/versions/5


In [8]:
import os
files = os.listdir(path)
print("Nom du fichier : ", files)

Nom du fichier :  ['netflix_titles.csv']


In [9]:
import pandas as pd

filename = f"{path}/{files[0]}"
df_initial = pd.read_csv(filename)

In [10]:
from tabulate import tabulate

print(tabulate(df_initial.head(10), headers='keys', tablefmt='psql'))

+----+-----------+---------+----------------------------------+-------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------+--------------------+----------------+----------+------------+---------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | show_id   | type    | title                            | director                      | cast                                                                                                                                                

In [11]:
# afficher le nombre de lignes dans le dataset
print(f"Nombre de lignes : {len(df_initial)}")

Nombre de lignes : 8807


In [12]:
# types des colonnes
print(df_initial.dtypes)

show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


In [13]:
# Convertir la colonne show_id en int en enlevant le préfixe 's' (plus simple pour gérer exactement le meme type de données en Cassandra et MySQL)
df_initial['show_id'] = df_initial['show_id'].str.replace('s', '').astype(int).astype(int)

df_initial['show_id'] = df_initial['show_id'].astype(int)
df_initial['release_year'] = df_initial['release_year'].astype(int)

df_initial['type'] = df_initial['title'].astype(str)
df_initial['title'] = df_initial['title'].astype(str)
df_initial['director'] = df_initial['director'].astype(str)
df_initial['cast'] = df_initial['cast'].astype(str)
df_initial['country'] = df_initial['country'].astype(str)
df_initial['date_added'] = df_initial['date_added'].astype(str)
df_initial['rating'] = df_initial['rating'].astype(str)
df_initial['duration'] = df_initial['duration'].astype(str)
df_initial['listed_in'] = df_initial['listed_in'].astype(str)
df_initial['description'] = df_initial['description'].astype(str)

In [14]:
# types des colonnes
print(df_initial.dtypes)

show_id          int64
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


In [15]:
# # est ce qu'il y a des valeurs manquantes ?
# print(df_initial.isna().sum())

In [16]:
# # remplir les valeurs manquantes : TODO supprimer cette cellule ?
# df_initial['director'] = df_initial['director'].fillna('')
# df_initial['cast'] = df_initial['cast'].fillna('')
# df_initial['country'] = df_initial['country'].fillna('')
# df_initial['date_added'] = df_initial['date_added'].fillna('')
# df_initial['rating'] = df_initial['rating'].fillna('')
# df_initial['duration'] = df_initial['duration'].fillna('')

# print(df_initial.isna().sum())
# TODO : vraiment utile ?

## CRUD : Create, Read, Update, Delete
- Create (*Insert*) : Insérer des données dans la base de données.
- Read (*Select*) : Récupérer des données.
- Update (*Update*) : Modifier des données existantes.
- Delete (*Delete*) : Effacer des données.

In [17]:
schema = """
CREATE TABLE IF NOT EXISTS shows (
    show_id INT PRIMARY KEY,
    title TEXT,
    type TEXT,
    director TEXT,
    cast TEXT,
    country TEXT,
    date_added TEXT,
    release_year INT,
    rating TEXT,
    duration TEXT,
    listed_in TEXT,
    description TEXT
);
"""

In [18]:
data_zoe_charlotte = {
    'show_id': [0],
    'title': ['Zoé & Charlotte'],
    'type': ['Movie'],
    'director': ['Christopher Nolan'],
    'cast': ['Leonardo DiCaprio, Joseph Gordon-Levitt'],
    'country': ['USA'],
    'date_added': ['2021-01-01'],
    'release_year': [2001],
    'rating': ['PG-13'],
    'duration': ['148 min'],
    'listed_in': ['Action, Sci-Fi'],
    'description': ['A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a CEO.']
}
df_zoe_charlotte = pd.DataFrame(data_zoe_charlotte)

### SQLite

In [19]:
import sqlite3

# Créer une base de données SQLite en mémoire
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Supprime la table si elle existe déjà
cursor.execute("DROP TABLE IF EXISTS shows")

# Créer une table pour les tests CRUD
cursor.execute(schema)


In [20]:
import time

# --- CREATE ---
def sqlite_insert(df):
    columns = ', '.join(df.columns)
    placeholders = ', '.join(['?'] * len(df.columns))
    insert_query = f"INSERT INTO shows ({columns}) VALUES ({placeholders})"

    start_time = time.time()
    for _, row in df.iterrows():
      cursor.execute(insert_query, tuple(row))
    conn.commit()

    create_time_sqlite = time.time() - start_time
    print(f"Create Time: {create_time_sqlite:.6f} seconds")
    return create_time_sqlite

# --- READ ---
def sqlite_read():
    start_time = time.time()
    cursor.execute("SELECT * FROM shows WHERE release_year > 2000")
    result = cursor.fetchall()
    read_time_sqlite = time.time() - start_time
    print(f"Read Time: {read_time_sqlite:.6f} seconds")
    print(f"Queried {len(result)} records")
    return read_time_sqlite

# --- UPDATE ---
def sqlite_update():
    start_time = time.time()
    cursor.execute("UPDATE shows SET rating='PG' WHERE release_year > 2000")
    conn.commit()
    update_time_sqlite = time.time() - start_time
    print(f"Update Time: {update_time_sqlite:.6f} seconds")
    return update_time_sqlite

# --- DELETE ---
def sqlite_delete():
    start_time = time.time()
    cursor.execute("DELETE FROM shows WHERE release_year > 2000")
    conn.commit()
    delete_time_sqlite = time.time() - start_time
    print(f"Delete Time: {delete_time_sqlite:.6f} seconds")
    return delete_time_sqlite

In [21]:
# --- CREATE ---
create_sqlite = sqlite_insert(df_zoe_charlotte)

# Vérifier le nombre de lignes insérées
cursor.execute('SELECT COUNT(*) FROM shows')
row = cursor.fetchone()
print(f"Total rows in 'shows' table: {row[0]}")

# --- READ ---
read_sqlite = sqlite_read()

# --- UPDATE ---
update_sqlite = sqlite_update()

# --- DELETE ---
delete_sqlite = sqlite_delete()

Create Time: 0.000231 seconds
Total rows in 'shows' table: 1
Read Time: 0.000124 seconds
Queried 1 records
Update Time: 0.000016 seconds
Delete Time: 0.000009 seconds


TODO rédaction : on le fait sur plus de données

In [22]:
# --- CREATE MASSIVE ---
create_sqlite_massive = sqlite_insert(df_initial)

# Vérifier le nombre de lignes insérées
cursor.execute('SELECT COUNT(*) FROM shows')
row = cursor.fetchone()
print(f"Total rows in 'shows' table: {row[0]}")

# --- READ MASSIVE ---
read_sqlite_massive = sqlite_read()

# --- UPDATE MASSIVE ---
update_sqlite_massive = sqlite_update()

# --- DELETE MASSIVE ---
delete_sqlite_massive = sqlite_delete()


Create Time: 0.102168 seconds
Total rows in 'shows' table: 8807
Read Time: 0.010810 seconds
Queried 8245 records
Update Time: 0.004174 seconds
Delete Time: 0.002798 seconds


### Cassandra

In [23]:
# vérifier l'état actuel du noeud Cassandra
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address    Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  127.0.0.1  304,79 KiB  16      100,0%            11c2e131-3642-40ab-a44b-5d5e95f42edb  rack1



In [24]:
from cassandra.cluster import Cluster

# Connexion à Cassandra
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

# Supprime la table avec keyspace si elle existe
session.execute("""
DROP TABLE IF EXISTS netflix.shows;
""")

# Créer un Keyspace avec un facteur de réplication de 3
session.execute("""
CREATE KEYSPACE IF NOT EXISTS netflix
WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")

# Changer le Keyspace courant pour 'netflix'
session.set_keyspace('netflix')

session.execute(schema)


TODO : pour l'instant on crée pas d'index mais on le fait après pour comparer

In [25]:
# --- CREATE ---
def cassandra_insert(df):
    insert_query = session.prepare("INSERT INTO shows (show_id, title, director, cast, country, date_added, release_year, rating, duration, listed_in, description) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)")

    start_time = time.time()
    for _, row in df.iterrows():
        session.execute(insert_query, (row['show_id'], row['title'], row['director'], row['cast'], row['country'], row['date_added'], row['release_year'], row['rating'], row['duration'], row['listed_in'], row['description']))
    create_time = time.time() - start_time

    print(f"Create Time: {create_time:.6f} seconds")

    return create_time

# --- READ ---
def cassandra_read():
    start_time = time.time()
    # On utilise ici ALLOW FILTERING car release_year ne fait pas partie de la clef primaire
    result = session.execute("SELECT * FROM shows WHERE release_year > 2000 ALLOW FILTERING")
    read_time = time.time() - start_time
    print(f"Read Time: {read_time:.6f} seconds")
    num_records = sum(1 for _ in result)  # Count rows using a generator expression
    print(f"Queried {num_records} records")
    return read_time

# --- UPDATE ---
def cassandra_update():
    # On doit faire select et update car realease_year ne fait pas partie de la clef primaire
    start_time = time.time()

    select_query = "SELECT show_id FROM shows WHERE release_year > 2000 ALLOW FILTERING;"
    result = session.execute(select_query)
    show_ids_to_update = [row.show_id for row in result]

    for show_id in show_ids_to_update:
        update_query = f"UPDATE shows SET rating = 'PG' WHERE show_id = {show_id};"
        session.execute(update_query)

    update_time = time.time() - start_time
    print(f"Update Time: {update_time:.6f} seconds")
    return update_time

# --- DELETE ---
def cassandra_delete():
  # Comme pour update
    start_time = time.time()

    select_query = "SELECT show_id FROM shows WHERE release_year > 2000 ALLOW FILTERING;"
    result = session.execute(select_query)
    show_ids_to_delete = [row.show_id for row in result]

    for show_id in show_ids_to_delete:
        delete_query = f"DELETE FROM shows WHERE show_id = {show_id};"
        session.execute(delete_query)

    delete_time = time.time() - start_time
    print(f"Delete Time: {delete_time:.6f} seconds")
    return delete_time

In [26]:
# --- CREATE ---
create_cassandra = cassandra_insert(df_zoe_charlotte)

# Vérifier le nombre de lignes insérées
result = session.execute('SELECT COUNT(*) FROM shows')
row = result.one()
print(f"Total rows in 'shows' table: {row[0]}")

# --- READ ---
read_cassandra = cassandra_read()

# --- UPDATE ---
update_cassandra = cassandra_update()

# --- DELETE ---
delete_cassandra = cassandra_delete()

Create Time: 0.001262 seconds
Total rows in 'shows' table: 1
Read Time: 0.000522 seconds
Queried 1 records
Update Time: 0.000523 seconds
Delete Time: 0.000458 seconds


In [27]:
# --- CREATE MASSIVE ---
create_cassandra_massive = cassandra_insert(df_initial)

# Vérifier le nombre de lignes insérées
result = session.execute('SELECT COUNT(*) FROM shows')
row = result.one()
print(f"Total rows in 'shows' table: {row[0]}")

# --- READ MASSIVE ---
read_cassandra_massive = cassandra_read()

# --- UPDATE MASSIVE ---
update_cassandra_massive = cassandra_update()

# --- DELETE MASSIVE ---
delete_cassandra_massive = cassandra_delete()


Create Time: 1.710411 seconds
Total rows in 'shows' table: 8807
Read Time: 0.052758 seconds
Queried 8245 records
Update Time: 1.421698 seconds
Delete Time: 1.349242 seconds


### Visualisation

TODO : rédiger

 pour faire de réelles stats on va faire les CRUD sur un nombre plus important de donnéer : insérer / lire / update / supprimer sur les données kaggle précédemment chargées

On a réalisé tout cela avec un seul replicaSet et sans index.

In [28]:
from re import X
import plotly.graph_objects as go

def affiche_times(set_1, barname_1, set_2, barname_2, titre):
  crud_operations = ['Insert', 'Read', 'Update', 'Delete']

  fig_simple = go.Figure(data=[
      go.Bar(name=barname_1, y=crud_operations, x=set_1, orientation='h'),
      go.Bar(name=barname_2, y=crud_operations, x=set_2, orientation='h')
  ])

  fig_simple.update_layout(barmode='group',
                          title=titre,
                          xaxis_title="Opération CRUD",
                          yaxis_title="Temps d'exécution (secondes)")

  fig_simple.show()

In [29]:
# enregistrer

sqlite_times_simple = [create_sqlite, read_sqlite, update_sqlite, delete_sqlite]
cassandra_times_simple = [create_cassandra, read_cassandra, update_cassandra, delete_cassandra]
sqlite_times_massive = [create_sqlite_massive, read_sqlite_massive, update_sqlite_massive, delete_sqlite_massive]
cassandra_times_massive = [create_cassandra_massive, read_cassandra_massive, update_cassandra_massive, delete_cassandra_massive]


In [30]:
affiche_times(sqlite_times_simple, 'SQLite', cassandra_times_simple, 'Cassandra', 'Comparaison des performances SQLite et Cassandra (Requêtes simples)' )

In [31]:
affiche_times(sqlite_times_massive, 'SQLite', cassandra_times_massive, 'Cassandra', 'Comparaison des performances SQLite et Cassandra (Requêtes massive)' )

TODO : rédiger commentairte sur le temps

pourquoi c'est peut etre normal ?

dire que ici cassandra c'est 1 replica set

et que y a pas d'index

on va faire sur notre dataset de 8k données

puis plus 

# TODO

## Comparaison des temps de Cassandra avec plusieurs replicaSet

In [32]:
def create_cassandra_with_replication(replication_factor):
    session.execute("DROP KEYSPACE IF EXISTS netflix")
    session.execute("DROP TABLE IF EXISTS netflix.shows")

    # TODO : Tester d'autres stratégies de réplication
    session.execute(f"""
    CREATE KEYSPACE netflix
    WITH REPLICATION = {{
        'class': 'SimpleStrategy', 
        'replication_factor': {replication_factor}
    }};
    """)
    # Utilisée pour les clusters en un seul datacenter. Chaque nœud du cluster conserve une copie des données basées 
    # sur la clé de partition et les réplique en fonction du facteur de réplication.
    session.set_keyspace('netflix')
    session.execute(schema)
    print(f"Keyspace créé avec replication_factor = {replication_factor}")

##### SimpleStrategy

Description : Réplique les données de manière linéaire sur les n nœuds du cluster, où n est le replication_factor.
Avantages :
Simple à configurer.
Adapté aux environnements de test ou aux clusters d'une seule région.
Inconvénients :
Pas optimisé pour les déploiements multi-régions.
Répartition des réplicas pas toujours équilibrée en cas de cluster complexe.

In [ ]:
import pandas as pd

# Liste des facteurs de réplication à tester
replication_factors = [1, 2, 3, 4, 5, ] # 6, 7, 8, 9, 10]

# Liste pour stocker les résultats
results = []

# Nombre d'itérations pour chaque facteur de réplication
iterations = 5

# Fonction pour effectuer les tests et obtenir le temps (il faut implémenter ces fonctions)
def perform_operations(rf):
    # Crée une base de données avec le facteur de réplication donné
    create_cassandra_with_replication(rf)
    
    # Mesure les temps pour les différentes opérations
    cassandra_time_create = cassandra_insert(df_initial)
    cassandra_time_read = cassandra_read()
    cassandra_time_update = cassandra_update()
    cassandra_time_delete = cassandra_delete()
    
    return cassandra_time_create, cassandra_time_read, cassandra_time_update, cassandra_time_delete

# Boucle sur chaque facteur de réplication
for rf in replication_factors:
    print(f"\nTest avec replication_factor = {rf} :")
    
    # Variables pour accumuler les temps
    total_create_time = 0
    total_read_time = 0
    total_update_time = 0
    total_delete_time = 0
    
    # Effectue l'opération 5 fois et calcule les temps
    for _ in range(iterations):
        cassandra_time_create, cassandra_time_read, cassandra_time_update, cassandra_time_delete = perform_operations(rf)
        
        total_create_time += cassandra_time_create
        total_read_time += cassandra_time_read
        total_update_time += cassandra_time_update
        total_delete_time += cassandra_time_delete
    
    # Calculer la moyenne des temps
    avg_create_time = total_create_time / iterations
    avg_read_time = total_read_time / iterations
    avg_update_time = total_update_time / iterations
    avg_delete_time = total_delete_time / iterations

    # Ajouter les résultats dans la liste
    results.append({
        'Replication Factor': rf,
        'Avg Insertion Time': avg_create_time,
        'Avg Read Time': avg_read_time,
        'Avg Update Time': avg_update_time,
        'Avg Delete Time': avg_delete_time
    })

    # TODO : avg, std...

# Convertir les résultats en DataFrame pour une analyse facile
results_df = pd.DataFrame(results)

# Afficher le DataFrame avec les résultats
print(results_df)



Test avec replication_factor = 1 :
Keyspace créé avec replication_factor = 1
Create Time: 1.779582 seconds
Read Time: 0.045865 seconds
Queried 8245 records
Update Time: 1.361120 seconds
Delete Time: 1.378874 seconds
Keyspace créé avec replication_factor = 1
Create Time: 1.690143 seconds
Read Time: 0.048973 seconds
Queried 8245 records
Update Time: 1.385077 seconds
Delete Time: 1.350375 seconds
Keyspace créé avec replication_factor = 1
Create Time: 1.704335 seconds
Read Time: 0.045742 seconds
Queried 8245 records
Update Time: 1.440612 seconds
Delete Time: 1.399207 seconds
Keyspace créé avec replication_factor = 1
Create Time: 1.795447 seconds
Read Time: 0.046558 seconds
Queried 8245 records
Update Time: 1.377523 seconds
Delete Time: 1.369297 seconds
Keyspace créé avec replication_factor = 1
Create Time: 1.772147 seconds
Read Time: 0.067597 seconds
Queried 8245 records
Update Time: 1.367903 seconds
Delete Time: 1.410278 seconds

Test avec replication_factor = 2 :
Keyspace créé avec repl

# TODO : faire tests en moyenne aussi sur 5 itérations
# pour sqlite 

In [34]:
crud_operations = ['Insert', 'Read', 'Update', 'Delete']

fig = go.Figure(data=[
    go.Bar(name='SQLite', y=crud_operations, x=sqlite_times_massive, orientation='h')  # SQLite bar
])

for rf in results_df['Replication Factor'].unique():
    rf_data = results_df[results_df['Replication Factor'] == rf]
    fig.add_trace(go.Bar(
        name=f"Cassandra (RF={rf})",
        y=crud_operations,
        x=[rf_data['Avg Insertion Time'].values[0],
           rf_data['Avg Read Time'].values[0],
           rf_data['Avg Update Time'].values[0],
           rf_data['Avg Delete Time'].values[0]],
        orientation='h'
    ))

fig.update_layout(barmode='group',
                  title="Comparaison des performances SQLite et Cassandra (Replica Sets)",
                  xaxis_title="Temps d'exécution (secondes)",
                  yaxis_title="Opération CRUD")

fig.show()

 SQLite est une base de données embarquée :
SQLite est une base de données légère et locale, conçue pour fonctionner sur des systèmes où les ressources sont limitées. Elle est souvent utilisée pour des bases de données petites à moyennes, et elle est optimisée pour être rapide dans des environnements où il n'y a pas de forte concurrence ou de distribution.
Comme SQLite est embarquée, elle utilise le même processus que ton application, ce qui peut réduire les frais généraux liés aux appels réseau et aux processus séparés nécessaires pour une base de données distribuée comme Cassandra.

Replication et gestion des nœuds dans Cassandra :
L'activation de la réplication dans Cassandra, même avec des facteurs de réplication faibles, introduit une complexité supplémentaire. Le fait que Cassandra doive gérer des copies des données sur plusieurs nœuds (et potentiellement attendre des confirmations d'écriture/lecture de ces nœuds) peut ralentir les performances, surtout pour des petites bases de données.
Le facteur de réplication dans Cassandra peut augmenter le temps de réponse des écritures et lectures, car les données doivent être synchronisées entre plusieurs nœuds. Dans une petite base de données, cela peut devenir un goulot d'étranglement inutile.


Coût de gestion des partitions et de la distribution :
Cassandra est conçue pour gérer de grandes quantités de données distribuées sur plusieurs machines. Si ta base de données est petite, Cassandra pourrait avoir un coût supplémentaire lié à la gestion des partitions et des nœuds, ce qui ralentirait les performances comparées à SQLite, qui n'a pas besoin de gérer cette distribution.


Caching et optimisation :
SQLite peut tirer parti du caching local pour accélérer les lectures, surtout lorsqu'il y a peu de données et que tout peut être chargé en mémoire.
Cassandra, d'autre part, peut nécessiter davantage de gestion des caches, de coordination entre les nœuds, et de répartition des données avant de pouvoir renvoyer les résultats d'une requête, ce qui peut rendre le processus plus lent pour des petites bases.

TODO : Effectuer des tests de charge : Augmenter le nombre de requêtes ou de transactions simultanées pour tester la scalabilité et la gestion des requêtes par Cassandra. Dans un scénario avec peu de données, SQLite peut sembler plus rapide, mais lorsque la charge augmente, Cassandra sera plus adapté pour gérer des performances à grande échelle

Dans Cassandra, la fragmentation horizontale (ou "sharding") se fait automatiquement à partir de la clé de partition que vous définissez dans vos tables. Cassandra distribue les données entre les nœuds en fonction des clés de partition et utilise la réplication pour garantir la résilience des données.

TODO vérifier valeur print c'est les memes que plot 

TODO : Le clustering est une autre méthode de distribution des données dans Cassandra, mais cela se fait au sein d'une partition (c'est-à-dire que les données ayant la même clé de partition seront rangées ensemble en fonction de la clé de clustering). Le choix d'une bonne clé de clustering permet d’optimiser la lecture des données dans Cassandra.

TODO CLUSTERING 

Utiliser une base de données plus grande : Cassandra est mieux adapté pour gérer de grandes quantités de données et pour tirer parti de son architecture distribuée.

On va donc augmenter notre jeu de données 

In [35]:
# TODO : augmenter le dataset pour tester la scalabilité
# TODO : BATCH
# TODO : INDEX
# TODO : REQUETES COMPLEXES
# TODO : JOINS
# TODO : REQUETES AVEC FILTRES

# Création d'un dataset plus grand

TODO rédaction

Création d'un dataset plus grand pour voir une réelle différence sur les opérations CRUD.  
Création nous même car les sites de création de dataset ne permettent de télécharger que 1000 lignes avec un compte gratuit.

In [36]:
df_initial.shape

(8807, 12)

In [67]:
import pandas as pd
import random

# Activer ou désactiver la génération de données supplémentaires
more_data = True

if more_data:
    # Charger le dataset initial
    original_df = df_initial  # Remplace par ton DataFrame de départ

    # Nombre total de lignes souhaitées
    num_rows = 30000

    # Fonction pour générer des colonnes supplémentaires à partir de données existantes
    def generate_large_column(original_column, size):
        return [random.choice(original_column) for _ in range(size)]

    # Fonction pour générer des IDs uniques
    def generate_unique_show_id(existing_ids, size):
        start_id = max(existing_ids) + 1 if existing_ids else 1
        return [f"{i}" for i in range(start_id, start_id + size)]

    # Générer les données pour chaque colonne
    generated_data = original_df.to_dict(orient='list')  # Convertir en dictionnaire de colonnes

    # Assurer l'unicité des IDs
    existing_ids = set(map(int, original_df["show_id"].tolist()))  # Obtenir les IDs existants
    new_ids = generate_unique_show_id(existing_ids, num_rows - len(original_df))
    generated_data["show_id"].extend(new_ids)

    # Générer les autres colonnes
    for column in original_df.columns:
        if column != "show_id":  # Ne pas regénérer la colonne show_id
            generated_data[column].extend(generate_large_column(original_df[column].tolist(), num_rows - len(original_df)))

    # Créer un DataFrame à partir des données générées
    large_df = pd.DataFrame(generated_data)

    # Supprimer les doublons éventuels (vérifie toutes les colonnes)
    large_df = large_df.drop_duplicates()

    # Remplacer les NaN explicitement pour chaque colonne
    for column in large_df.columns:
        if large_df[column].isnull().any():
            if large_df[column].dtype == "object":
                large_df[column].fillna("Unknown", inplace=True)  # Texte : remplacer par 'Unknown'
            else:
                large_df[column].fillna(0, inplace=True)  # Numérique : remplacer par 0

    # Supprimer les doublons éventuels (vérifie toutes les colonnes)
    large_df = large_df.drop_duplicates()

    # Vérifie qu'il n'y a pas de doublons dans 'show_id'
    if large_df["show_id"].duplicated().any():
        print("Attention : Des doublons existent dans la colonne 'show_id'.")
    else:
        print("Les IDs sont uniques.")

    # Sauvegarder le dataset dans un fichier CSV
    output_file = "netflix_titles_large.csv"
    large_df.to_csv(output_file, index=False)
    print(f"Dataset généré avec {len(large_df)} lignes (lignes dupliquées supprimées) et exporté dans {output_file}.")


Les IDs sont uniques.
Dataset généré avec 30000 lignes (lignes dupliquées supprimées) et exporté dans netflix_titles_large.csv.


In [68]:
large_df.shape

(30000, 12)

In [69]:
large_df.isna().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [73]:
# Convertir la colonne show_id en int en enlevant le préfixe 's' (plus simple pour gérer exactement le meme type de données en Cassandra et MySQL)
large_df['show_id'] = large_df['show_id'].astype(int)

large_df['show_id'] = large_df['show_id'].astype(int)
large_df['release_year'] = large_df['release_year'].astype(int)

large_df['type'] = large_df['title'].astype(str)
large_df['title'] = large_df['title'].astype(str)
large_df['director'] = large_df['director'].astype(str)
large_df['cast'] = large_df['cast'].astype(str)
large_df['country'] = large_df['country'].astype(str)
large_df['date_added'] = large_df['date_added'].astype(str)
large_df['rating'] = large_df['rating'].astype(str)
large_df['duration'] = large_df['duration'].astype(str)
large_df['listed_in'] = large_df['listed_in'].astype(str)
large_df['description'] = large_df['description'].astype(str)

In [74]:
# Supprime la table si elle existe déjà
cursor.execute("""
DROP TABLE IF EXISTS shows;
""")

# Créer une table pour les tests CRUD
cursor.execute(schema)

In [ ]:
# affiche ce qu'il y a dans la table shows
cursor.execute("SELECT * FROM shows")
result = cursor.fetchall()
print(result)

[]


In [ ]:
# TODO : faire tests en moyenne aussi sur 5 itérations

# --- CREATE MASSIVE ---
create_sqlite_massive = sqlite_insert(large_df)

# Vérifier le nombre de lignes insérées
cursor.execute('SELECT COUNT(*) FROM shows')
row = cursor.fetchone()
print(f"Total rows in 'shows' table: {row[0]}")

# --- READ MASSIVE ---
read_sqlite_massive = sqlite_read()

# --- UPDATE MASSIVE ---
update_sqlite_massive = sqlite_update()

# --- DELETE MASSIVE ---
delete_sqlite_massive = sqlite_delete()

Create Time: 0.386195 seconds
Total rows in 'shows' table: 30000
Read Time: 0.035644 seconds
Queried 28058 records
Update Time: 0.013219 seconds
Delete Time: 0.010089 seconds


In [77]:
import pandas as pd

# Liste des facteurs de réplication à tester
replication_factors = [1, 2, 3, 4, 5, ] #6, 7, 8, 9, 10]

# Liste pour stocker les résultats
results = []

# Nombre d'itérations pour chaque facteur de réplication
iterations = 5

# Fonction pour effectuer les tests et obtenir le temps (il faut implémenter ces fonctions)
def perform_operations(rf):
    # Crée une base de données avec le facteur de réplication donné
    create_cassandra_with_replication(rf)
    
    # Mesure les temps pour les différentes opérations
    cassandra_time_create = cassandra_insert(large_df)
    cassandra_time_read = cassandra_read()
    cassandra_time_update = cassandra_update()
    cassandra_time_delete = cassandra_delete()
    
    return cassandra_time_create, cassandra_time_read, cassandra_time_update, cassandra_time_delete

# Boucle sur chaque facteur de réplication
for rf in replication_factors:
    print(f"\nTest avec replication_factor = {rf} :")
    
    # Variables pour accumuler les temps
    total_create_time = 0
    total_read_time = 0
    total_update_time = 0
    total_delete_time = 0
    
    # Effectue l'opération 5 fois et calcule les temps
    for _ in range(iterations):
        cassandra_time_create, cassandra_time_read, cassandra_time_update, cassandra_time_delete = perform_operations(rf)
        
        total_create_time += cassandra_time_create
        total_read_time += cassandra_time_read
        total_update_time += cassandra_time_update
        total_delete_time += cassandra_time_delete
    
    # Calculer la moyenne des temps
    avg_create_time = total_create_time / iterations
    avg_read_time = total_read_time / iterations
    avg_update_time = total_update_time / iterations
    avg_delete_time = total_delete_time / iterations

    # Ajouter les résultats dans la liste
    results.append({
        'Replication Factor': rf,
        'Avg Insertion Time': avg_create_time,
        'Avg Read Time': avg_read_time,
        'Avg Update Time': avg_update_time,
        'Avg Delete Time': avg_delete_time
    })

# Convertir les résultats en DataFrame pour une analyse facile
results_df = pd.DataFrame(results)

# Afficher le DataFrame avec les résultats
print(results_df)



Test avec replication_factor = 1 :
Keyspace créé avec replication_factor = 1
Create Time: 6.119463 seconds
Read Time: 0.050231 seconds
Queried 28058 records
Update Time: 4.983294 seconds
Delete Time: 4.647389 seconds
Keyspace créé avec replication_factor = 1
Create Time: 6.892102 seconds
Read Time: 0.047462 seconds
Queried 28058 records
Update Time: 4.832289 seconds
Delete Time: 4.754344 seconds
Keyspace créé avec replication_factor = 1
Create Time: 6.444077 seconds
Read Time: 0.058396 seconds
Queried 28058 records
Update Time: 4.738052 seconds
Delete Time: 4.846134 seconds
Keyspace créé avec replication_factor = 1
Create Time: 6.361893 seconds
Read Time: 0.052594 seconds
Queried 28058 records
Update Time: 4.796468 seconds
Delete Time: 4.784129 seconds
Keyspace créé avec replication_factor = 1
Create Time: 5.626514 seconds
Read Time: 0.045238 seconds
Queried 28058 records
Update Time: 4.668402 seconds
Delete Time: 4.626672 seconds

Test avec replication_factor = 2 :
Keyspace créé avec

In [79]:
sqlite_times_massive = [create_sqlite_massive, read_sqlite_massive, update_sqlite_massive, delete_sqlite_massive]

crud_operations = ['Insert', 'Read', 'Update', 'Delete']

fig = go.Figure(data=[
    go.Bar(name='SQLite', y=crud_operations, x=sqlite_times_massive, orientation='h')  # SQLite bar
])

for rf in results_df['Replication Factor'].unique():
    rf_data = results_df[results_df['Replication Factor'] == rf]
    fig.add_trace(go.Bar(
        name=f"Cassandra (RF={rf})",
        y=crud_operations,
        x=[rf_data['Avg Insertion Time'].values[0],
           rf_data['Avg Read Time'].values[0],
           rf_data['Avg Update Time'].values[0],
           rf_data['Avg Delete Time'].values[0]],
        orientation='h'
    ))

fig.update_layout(barmode='group',
                  title="Comparaison des performances SQLite et Cassandra (Replica Sets) sur le dataset massif",
                  xaxis_title="Temps d'exécution (secondes)",
                  yaxis_title="Opération CRUD")

fig.show()

# Comparaison avec / sans index

In [ ]:
def cassandra_create_indexes(columns):
    start_time = time.time()
    for column in columns:
        index_query = f"CREATE INDEX {column}_idx ON shows ({column});"
        session.execute(index_query)
    cassandra_time = time.time() - start_time
    return cassandra_time


def sqlite_create_indexes(columns):
    start_time = time.time()
    for column in columns:
        # Créer une requête SQL pour chaque colonne
        index_query = f"CREATE INDEX IF NOT EXISTS {column}_idx ON shows ({column});"
        cursor.execute(index_query)
    sqlite_time = time.time() - start_time
    return sqlite_time

In [ ]:
columns_to_index = ["release_year"]
cassandra_time_index = cassandra_create_indexes(columns_to_index)
mysql_time_index = sqlite_create_indexes(columns_to_index)

print(f"Indexation time: Cassandra {cassandra_time_index:.5f}s, SQLite {mysql_time_index:.5f}s")

In [ ]:
# Supprime la table si elle existe déjà
cursor.execute("""
DROP TABLE IF EXISTS shows;
""")

# Créer une table pour les tests CRUD
cursor.execute(schema)

In [ ]:
# --- CREATE MASSIVE ---
create_sqlite_massive_index = sqlite_insert(large_df)

# Vérifier le nombre de lignes insérées
cursor.execute('SELECT COUNT(*) FROM shows')
row = cursor.fetchone()
print(f"Total rows in 'shows' table: {row[0]}")

# --- READ MASSIVE ---
read_sqlite_massive_index = sqlite_read()

# --- UPDATE MASSIVE ---
update_sqlite_massive_index = sqlite_update()

# --- DELETE MASSIVE ---
delete_sqlite_massive_index = sqlite_delete()

In [ ]:
# # TODO : 
# import matplotlib.pyplot as plt
# import numpy as np

# # Données simulées
# replication_factors = [1, 2, 3, 4, 5]
# insertion_mean = [0.3, 0.35, 0.4, 0.45, 0.5]
# insertion_std = [0.02, 0.03, 0.04, 0.05, 0.06]

# # Graphique
# plt.figure(figsize=(10, 6))
# plt.errorbar(replication_factors, insertion_mean, yerr=insertion_std, fmt='o-', capsize=5, label="Insertion Time")

# plt.title("Performance en fonction du Replication Factor")
# plt.xlabel("Replication Factor")
# plt.ylabel("Time (s)")
# plt.legend()
# plt.grid(True)
# plt.show()


# # Données simulées
# x = np.array(replication_factors)
# insert_mean = np.array(insertion_mean)
# insert_std = np.array(insertion_std)

# plt.figure(figsize=(10, 6))
# plt.plot(x, insert_mean, 'o-', label="Insertion Time")
# plt.fill_between(x, insert_mean - insert_std, insert_mean + insert_std, alpha=0.2, label="Insertion Uncertainty")

# plt.title("Performance avec zones d'incertitude")
# plt.xlabel("Replication Factor")
# plt.ylabel("Time (s)")
# plt.legend()
# plt.grid(True)
# plt.show()


In [ ]:
# plt.figure(figsize=(10, 6))
# plt.plot(cassandra_times_update, label="Cassandra Update", marker='o')
# plt.plot(mysql_times_update, label="MySQL Update", marker='x')
# plt.xlabel("Itérations")
# plt.ylabel("Temps (secondes)")
# plt.title("Benchmark des mises à jour (Update) entre Cassandra et MySQL")
# plt.legend()
# plt.grid()
# plt.show()

In [ ]:
# plt.figure(figsize=(10, 6))
# plt.plot(cassandra_times_delete, label="Cassandra Delete", marker='o')
# plt.plot(mysql_times_delete, label="MySQL Delete", marker='x')
# plt.xlabel("Itérations")
# plt.ylabel("Temps (secondes)")
# plt.title("Benchmark des suppressions (Delete) entre Cassandra et MySQL")
# plt.legend()
# plt.grid()
# plt.show()


TODO : index secondaire ?

Indexation des colonnes utilisées dans les filtres, jointures et ordonnancement (ORDER BY).  
Seulement ces colonnes sinon le coût de stockage est trop élevé pour rien.

In [ ]:
cassandra_time_create_idx, mysql_time_create_idx = recreate_tables_with_indexes()
print(f"Create time: Cassandra {cassandra_time_create_idx:.5f}s, MySQL {mysql_time_create_idx:.5f}s")

In [ ]:
# Vérification des index
index_query = "SELECT * FROM system_schema.indexes WHERE keyspace_name = 'netflix';"
rows = cassandra_session.execute(index_query)
print("Index existants dans Cassandra :")
for row in rows:
    print(row)

index_query_mysql = "SHOW INDEX FROM shows;"
with mysql_engine.connect() as conn:
    conn.execute(text("USE TDLE;"))
    result = conn.execute(text(index_query_mysql))
    print("Index existants dans MySQL :")
    for row in result:
        print(row)

### Résultats CRUD après indexation

In [ ]:
crud_operations = ['Create', 'Read', 'Update', 'Delete']

cassandra_times_after = [
    cassandra_time_create_idx,
    sum(cassandra_times_read) / len(cassandra_times_read),
    sum(cassandra_times_update) / len(cassandra_times_update),
    sum(cassandra_times_delete) / len(cassandra_times_delete),
]

mysql_times_after = [
    mysql_time_create_idx,
    sum(mysql_times_read) / len(mysql_times_read),
    sum(mysql_times_update) / len(mysql_times_update),
    sum(mysql_times_delete) / len(mysql_times_delete),
]

fig = go.Figure()
fig.add_trace(go.Bar(x=crud_operations, y=cassandra_times_after, name='Cassandra (Après Indexation)', text=[f"{t:.2f}s" for t in cassandra_times_after], textposition='auto'))
fig.add_trace(go.Bar(x=crud_operations, y=mysql_times_after, name='MySQL (Après Indexation)', text=[f"{t:.2f}s" for t in mysql_times_after], textposition='auto'))

fig.update_layout(title="Comparaison des performances CRUD après indexation entre Cassandra et MySQL", xaxis_title="Opérations CRUD", yaxis_title="Temps d'exécution (secondes)", barmode='group', template='plotly_white', legend=dict(title="Système de base de données"))

fig.show()


In [ ]:
crud_operations = ['Create', 'Read', 'Update', 'Delete']

cassandra_times = [
    cassandra_time_create,
    sum(cassandra_times_read) / len(cassandra_times_read),
    sum(cassandra_times_update) / len(cassandra_times_update),
    sum(cassandra_times_delete) / len(cassandra_times_delete),
]

cassandra_times_after = [
    cassandra_time_create_idx,
    sum(cassandra_time_read_idx) / len(cassandra_time_read_idx),
    sum(cassandra_time_update_idx) / len(cassandra_time_update_idx),
    sum(cassandra_time_delete_idx) / len(cassandra_time_delete_idx),
]

mysql_times = [
    mysql_time_create,
    sum(mysql_times_read) / len(mysql_times_read),
    sum(mysql_times_update) / len(mysql_times_update),
    sum(mysql_times_delete) / len(mysql_times_delete),
]

mysql_times_after = [
    mysql_time_create_idx,
    sum(mysql_time_read_idx) / len(mysql_time_read_idx),
    sum(mysql_time_update_idx) / len(mysql_time_update_idx),
    sum(mysql_time_delete_idx) / len(mysql_time_delete_idx),
]

# Cassandra
fig_cassandra = go.Figure()

fig_cassandra.add_trace(go.Bar(x=crud_operations, y=cassandra_times, name='Cassandra (Avant Indexation)', text=[f"{t:.2f}s" for t in cassandra_times], textposition='auto'))
fig_cassandra.add_trace(go.Bar(x=crud_operations, y=cassandra_times_after, name='Cassandra (Après Indexation)', text=[f"{t:.2f}s" for t in cassandra_times_after], textposition='auto'))

fig_cassandra.update_layout(title="Comparaison des performances CRUD de Cassandra (Avant vs Après Indexation)", xaxis_title="Opérations CRUD", yaxis_title="Temps d'exécution (secondes)", barmode='group', template='plotly_white')
fig_cassandra.show()

# MySQL
fig_mysql = go.Figure()

fig_mysql.add_trace(go.Bar(x=crud_operations, y=mysql_times, name='MySQL (Avant Indexation)', text=[f"{t:.2f}s" for t in mysql_times], textposition='auto'))
fig_mysql.add_trace(go.Bar(x=crud_operations, y=mysql_times_after, name='MySQL (Après Indexation)', text=[f"{t:.2f}s" for t in mysql_times_after], textposition='auto'))

fig_mysql.update_layout( title="Comparaison des performances CRUD de MySQL (Avant vs Après Indexation)", xaxis_title="Opérations CRUD", yaxis_title="Temps d'exécution (secondes)", barmode='group', template='plotly_white')
fig_mysql.show()


In [ ]:

# TODO :  Analyser les résultats


# TODO : Tester avec des requêtes plus complexes
# TODO : Utiliser un dataset plus grand
# TODO : Ajouter des colonnes à indexer
# Tous ces TODO n'ont rien changé.

## Optimisation par réplication/partitionnement

In [ ]:
# Recréation des tables pour avoir toutes les données
def recreate_tables():
    create_table_cassandra()

    global mysql_engine
    mysql_engine = create_table_mysql()

    cassandra_time_create = cassandra_insert(df)
    mysql_time_create = mysql_insert(df)

    return cassandra_time_create#, mysql_time_create

cassandra_time_create = recreate_tables()#, mysql_time_create

### Cassandra - Réplication

In [ ]:
# ZOE OK
# def create_keyspace_with_replication(replication_factor):
#     cassandra_session.execute(f"DROP KEYSPACE IF EXISTS netflix;")

#     # TODO : Tester d'autres stratégies de réplication
#     cassandra_session.execute(f"""
#     CREATE KEYSPACE netflix
#     WITH REPLICATION = {{
#         'class': 'SimpleStrategy',
#         'replication_factor': {replication_factor}
#     }};
#     """)
#     cassandra_session.set_keyspace('netflix')
#     print(f"Keyspace créé avec replication_factor = {replication_factor}")

### Cassandra - Test du CRUD

In [ ]:
# ZOE OK
# replication_factors = [1, 2, 3] #,4, 5, 6, 7, 8, 9, 10]
# results = []

# for rf in replication_factors:
#     print(f"\nTest avec replication_factor = {rf} :")
#     create_keyspace_with_replication(rf)
#     create_table_cassandra()
#     cassandra_time_create = cassandra_insert(df)
#     print(f"Insertion time: {cassandra_time_create:.5f}s")
#     cassandra_time_read = cassandra_read_benchmark()
#     #print(f" - Read time: {cassandra_time_read:.5f}s")
#     cassandra_time_update = cassandra_update_benchmark()
#     #print(f" - Update time: {cassandra_time_update:.5f}s")
#     cassandra_time_delete = cassandra_delete_benchmark_with_reset_from_df()
#     #print(f" - Delete time: {cassandra_time_delete:.5f}s")

#     results.append({
#         'Replication Factor': rf,
#         'Insertion Time': cassandra_time_create,
#         'Read Time': cassandra_time_read,
#         'Update Time': cassandra_time_update,
#         'Delete Time': cassandra_time_delete
#     })

# results_df = pd.DataFrame(results)

### Cassandra - Résultats

In [ ]:
average_times = results_df.iloc[:, 1:].mean()

for col in ['Insertion Time', 'Read Time', 'Update Time', 'Delete Time']:
    results_df[col] = results_df[col].apply(lambda x: sum(x)/len(x) if isinstance(x, list) else x)

average_times = results_df.iloc[:, 1:].mean()

fig = go.Figure()


for rf in results_df['Replication Factor']:
    values = results_df[results_df['Replication Factor'] == rf].iloc[0, 1:].values
    values = [float(v) for v in values]

    fig.add_trace(go.Bar(x=['Insertion Time', 'Read Time', 'Update Time', 'Delete Time'], y=values, name=f'Replication Factor {rf}', text=[f"{v:.3f}s" for v in values], textposition='auto'))

fig.update_layout(title="Temps d'exécution des opérations CRUD par facteur de réplication", xaxis_title="Opérations CRUD", yaxis_title="Temps d'exécution (secondes)", barmode='group', template='plotly_white', legend=dict(title="Replication Factor"))

fig.show()

# Fin du notebook

In [ ]:
# Tout fermer proprement
session.shutdown() # Fermer la connexion Cassandra
cluster.shutdown() # Fermer la connexion Cassandra
mysql_engine.dispose() # Fermer la connexion MySQL

# Sources

- http://www-igm.univ-mlv.fr/~dr/XPOSE2010/Cassandra/modele.html